Installing the Julep Client

In [2]:
!pip install julep -U --quiet

In [13]:
locations = [ "Noida", "Delhi", "New York", "London", "Paris", "Tokyo", "Sydney"]

In [1]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### creating julep client with the api key


In [2]:
from julep import Client
import os

JULEP_API_KEY = os.environ["JULEP_API_KEY"]

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [3]:
# Defining the agent
name = "Saviez Vous"
about = "A city guide like agent that tells you 3 fun facts about cities using wikipedia!."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [4]:
import yaml

# Defining the task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Julep City Fun Facts
description: A task that gives you 3 fun facts about a city using Wikipedia.

########################################################
####################### INPUT SCHEMA ##################
########################################################                       

input_schema:
  type: object
  properties:
    locations:
      type: array
      items:
        type: string
      description: The list of city names.

########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: wikipedia
  type: integration
  integration:
    provider: wikipedia

########################################################
####################### MAIN WORKFLOW ##########################
########################################################

main:
# Step 0: Search Wikipedia summary for each city
- over: $ steps[0].input.locations
  map:
    tool: wikipedia
    arguments:
      query: $_

# Step 1: Generate 3 cool facts from each city description
- over: $ steps[0].output
                          
  map:
    prompt:
    - role: system
      content: |
        You are an assistant that extracts interesting “Did-you-know?” style facts from a city description.
        Given a text about a city, pick three surprising or cool facts (historical events, landmarks, legends).
        Return the facts as a friendly list of three “Did-you-know?” blurbs.
    - role: user
      content: |-
        Here is a description of a city:
        "{{_}}"
        Extract three cool facts for a visitor.

# Step 2: Zip city names and their extracted facts into a list of tuples [(city, facts)]
- evaluate:
    zipped: |-
      $ list(zip(steps[0].input.locations, steps[1].output))

# Step 3: Format a friendly "Did you know?" paragraph per city
- over: $ _['zipped']
  map:
    prompt:
    - role: system
      content: |-
        Format the city and facts into a friendly paragraph with the heading "Did you know about [City]?"

    - role: user
      content: |-
        City: "{{_[0]}}"
        Facts:
        "{{_[1]}}"

# Step 4: Join everything into one final message
- evaluate:
    output: $ '\\n\\n'.join(str(item) for item in _)                      
""")

In [5]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [ ]:
execution = client.executions.create(
    task_id=task.id,
    input={
         "locations": locations,    

         #[ "Noida", "Delhi", "New York", "London", "Paris", "Tokyo", "Sydney"] by default
            # you can change the locations to any city you want from the first cell
    }
)

print("Started an execution. Execution ID:", execution.id)

Started an execution. Execution ID: 06845e79-04d7-7cde-8000-3fceaec0c28f


## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [8]:
import time

execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

execution = client.executions.get(execution.id)

if 'final_plan' in execution.output:
    print(execution.output['final_plan'])
else:
    print(execution.output)


Execution status:  running
--------------------------------------------------
Execution status:  succeeded
--------------------------------------------------
{'output': '{\'id\': \'chatcmpl-BgGCFUIeW3Z694hcYPF2IzOjtOuRh\', \'created\': 1749411735, \'model\': \'gpt-4o-2024-08-06\', \'object\': \'chat.completion\', \'system_fingerprint\': \'fp_07871e2ad8\', \'choices\': [{\'finish_reason\': \'stop\', \'index\': 0, \'message\': {\'content\': "**Did you know about Noida?**\\n\\nNoida, an acronym for New Okhla Industrial Development Authority, is a bustling city located in the northern Indian state of Uttar Pradesh. As a part of the National Capital Region, Noida is known for its modern infrastructure and rapid urban development. The city offers a mix of vibrant life and serene spaces, with a plethora of shopping malls, entertainment hubs, and green parks. With its world-class roads, vast green belts, and robust transport system, including the Delhi Metro, Noida is an attractive destination

In [10]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'locations': ['Noida', 'Delhi', 'New York', 'London', 'Paris', 'Tokyo', 'Sydney']}
--------------------------------------------------
Transition type:  init_branch
Transition output:  Noida
--------------------------------------------------
Transition type:  init_branch
Transition output:  Delhi
--------------------------------------------------
Transition type:  init_branch
Transition output:  New York
--------------------------------------------------
Transition type:  init_branch
Transition output:  Sydney
--------------------------------------------------
Transition type:  init_branch
Transition output:  Tokyo
--------------------------------------------------
Transition type:  init_branch
Transition output:  London
--------------------------------------------------
Transition type:  init_branch
Transition output:  Paris
--------------------------------------------------
Transition type:  finish_branch
Transition output:  {'documents': []

In [14]:
final_transition = transitions[0]
final_transition.output

{'output': '{\'id\': \'chatcmpl-BgGCFUIeW3Z694hcYPF2IzOjtOuRh\', \'created\': 1749411735, \'model\': \'gpt-4o-2024-08-06\', \'object\': \'chat.completion\', \'system_fingerprint\': \'fp_07871e2ad8\', \'choices\': [{\'finish_reason\': \'stop\', \'index\': 0, \'message\': {\'content\': "**Did you know about Noida?**\\n\\nNoida, an acronym for New Okhla Industrial Development Authority, is a bustling city located in the northern Indian state of Uttar Pradesh. As a part of the National Capital Region, Noida is known for its modern infrastructure and rapid urban development. The city offers a mix of vibrant life and serene spaces, with a plethora of shopping malls, entertainment hubs, and green parks. With its world-class roads, vast green belts, and robust transport system, including the Delhi Metro, Noida is an attractive destination for both residents and businesses. Whether you\'re interested in technology, education, or culture, Noida has something remarkable to offer, representing a d

In [12]:
import ast

output_str = final_transition.output['output']
cleaned_str = "[" + output_str.replace("}\n\n{", "},{") + "]"

parsed_outputs = ast.literal_eval(cleaned_str)

final_text = "\n\n".join(
    choice.get('message', {}).get('content', '')
    for completion in parsed_outputs
    for choice in completion.get('choices', [])
)
print(final_text)


**Did you know about Noida?**

Noida, an acronym for New Okhla Industrial Development Authority, is a bustling city located in the northern Indian state of Uttar Pradesh. As a part of the National Capital Region, Noida is known for its modern infrastructure and rapid urban development. The city offers a mix of vibrant life and serene spaces, with a plethora of shopping malls, entertainment hubs, and green parks. With its world-class roads, vast green belts, and robust transport system, including the Delhi Metro, Noida is an attractive destination for both residents and businesses. Whether you're interested in technology, education, or culture, Noida has something remarkable to offer, representing a dynamic blend of progress and tradition.

**Did you know about Delhi?**

Delhi, the vibrant capital city of India, is a cultural and historical treasure trove! Known for its rich history, it is home to stunning architectures like the Red Fort, Qutub Minar, and India Gate. Delhi seamlessly bl

## Related Concepts

- [Agents](/concepts/agents)
- [Tasks](/concepts/tasks)
- [Tools](/concepts/tools)
